# Minimum Fisher

In [ ]:
push!(LOAD_PATH, "./src")
using RegularizedSystems
using VSTom
using PyPlot
using ProgressMeter

In [ ]:
function make_synthetic_data{T<:Real}(A::AbstractArray{T,2},dist::AbstractArray{T,2},n::T,b::T,dE::T,dP::T)
    y = (dE*dP) * (A*vec(dist))
    
    scale = n*mean(sqrt(y))
    yerr = scale .* max(sqrt(y),sqrt(b)) .* randn(length(y))
    
    noisy_spectrum = y .+ yerr
    noisy_background = b .+ scale*sqrt(b).*randn(length(y))
    
    scale_est = n*mean(sqrt(abs(noisy_spectrum)))
    
    err = scale_est .* max(sqrt(abs(noisy_spectrum)),sqrt(abs(noisy_background)))
    
    return noisy_spectrum, err
end

## Reconstruct distributions on 20x20 grid

In [ ]:
nenergy = 20
npitch = 20

test_dists = read_ncdf("data/test_distributions.cdf");
test_data = read_ncdf("data/test_data.cdf")
wf   = read_ncdf("data/reduced_synthetic_transfer_matrix.cdf");

A_full = wf["transfer_matrix"]
energy_full = wf["energy"]
pitch_full  = wf["pitch"]

dE_full = abs(energy_full[2] - energy_full[1])
dP_full = abs(pitch_full[2] - pitch_full[1])
energy = linspace(20.0,maximum(energy_full),nenergy)
pitch = linspace(extrema(pitch_full)...,npitch)
A = resize_transfer_matrix(A_full,energy_full,pitch_full,energy,pitch);
dE = abs(energy[2]-energy[1])
dP = abs(pitch[2]-pitch[1]);
L1E,L1p = gradEP(energy,pitch);

In [ ]:
transp_full = test_dists["transp"]
transp = bilinear(transp_full,energy_full,pitch_full,energy,pitch)
transp_vec = vec(transp)
nr,nc = size(A)
spec, err = make_synthetic_data(A_full,transp_full,0.5,1.25e14,dE_full,dP_full)
RF = FisherFunctional(A,spec,err,(L1E,L1p))
alpha = lcurve(RF,log_alpha_range=(0,20),nseeds=25,doplot=true,kwargs={:maxiter=>5,:verbose=>false},filename="mf_transp_k=0.5_lcurve.eps")

In [ ]:
function mf_analysis(A_full,dist_full,dE_full,dP_full,A,dE,dP,L1E,L1p,n;num=20)
    nr,nc = size(A)
    x = zeros(nc,num)
    for i=1:num
        spec,err = make_synthetic_data(A_full,dist_full,n,1.25e14,dE_full,dP_full)
        RF = FisherFunctional(A,spec,err,(L1E,L1p))
        alpha = lcurve(RF,log_alpha_range=(0,20),nseeds=20,kwargs={:maxiter=>5,:verbose=>false})
        sol = minimize(RF,alpha,maxiter=5)
        x[:,i] = max(sol.x,0)/(dE*dP)
    end
    
    return x
end

In [ ]:
transp_full = test_dists["transp"]
transp = bilinear(transp_full,energy_full,pitch_full,energy,pitch)
transp_vec = vec(transp)
nr,nc = size(A)
t_x_mean = zeros(nc,20)
t_x_var = zeros(nc,20)
t_x_bias = zeros(nc,20)
t_x_mse = zeros(nc,20)

t_d_mean = zeros(20)
t_d_var = zeros(20)
t_d_bias = zeros(20)
t_d_mse = zeros(20)

@showprogress for (i,n) in enumerate(0.05:0.05:1.0)
    x_samples = mf_analysis(A_full,transp_full,dE_full,dP_full,A,dE,dP,L1E,L1p,n)

    t_x_mean[:,i] = squeeze(mean(x_samples,2),2)
    t_x_var[:,i] = squeeze(var(x_samples,2),2)
    t_x_bias[:,i] = t_x_mean[:,i] - transp_vec
    t_x_mse[:,i] = t_x_bias[:,i].^2 + t_x_var[:,i]
    
    t_d_mean[i] = mean(sum(x_samples,1)*dE*dP)
    t_d_var[i] = var(sum(x_samples,1)*dE*dP)
    t_d_bias[i] = t_d_mean[i] - 1e12
    t_d_mse[i] = t_d_bias[i]^2 + t_d_var[i]
end

In [ ]:
x_samples = mf_analysis(A_full,transp_full,dE_full,dP_full,A,dE,dP,L1E,L1p,1.0)
i=20
t_x_mean[:,i] = squeeze(mean(x_samples,2),2)
t_x_var[:,i] = squeeze(var(x_samples,2),2)
t_x_bias[:,i] = t_x_mean[:,i] - transp_vec
t_x_mse[:,i] = t_x_bias[:,i].^2 + t_x_var[:,i]

t_d_mean[i] = mean(sum(x_samples,1)*dE*dP)
t_d_var[i] = var(sum(x_samples,1)*dE*dP)
t_d_bias[i] = t_d_mean[i] - 1e12
t_d_mse[i] = t_d_bias[i]^2 + t_d_var[i]

In [ ]:
using NetCDF
# Write synthetic data to file
fname = "data/mf_sampled_transp.cdf"

isfile(fname) && rm(fname)

nrows_id = NcDim("nrows",20*20)
ncols_id = NcDim("ncols",20)
nen_id = NcDim("nenergy",20)
np_id = NcDim("npitch",20)

x_mean_varid = NcVar("x_mean",[nrows_id,ncols_id])
x_var_varid = NcVar("x_var",[nrows_id,ncols_id])
x_bias_varid = NcVar("x_bias",[nrows_id,ncols_id])
x_mse_varid = NcVar("x_mse",[nrows_id,ncols_id])

d_mean_varid = NcVar("d_mean",ncols_id)
d_var_varid = NcVar("d_var",ncols_id)
d_bias_varid = NcVar("d_bias",ncols_id)
d_mse_varid = NcVar("d_mse",ncols_id)

energy_varid = NcVar("energy",nen_id)
pitch_varid = NcVar("pitch",nen_id)
elevel_varid = NcVar("error_level",ncols_id)

ncid = NetCDF.create(fname,[x_mean_varid,x_var_varid,x_bias_varid,x_mse_varid,
    d_mean_varid,d_var_varid,d_bias_varid,d_mse_varid,energy_varid,pitch_varid,elevel_varid],mode=NC_CLASSIC_MODEL)

NetCDF.putvar(ncid,"x_mean",t_x_mean)
NetCDF.putvar(ncid,"x_var",t_x_var)
NetCDF.putvar(ncid,"x_bias",t_x_bias)
NetCDF.putvar(ncid,"x_mse",t_x_mse)

NetCDF.putvar(ncid,"d_mean",t_d_mean)
NetCDF.putvar(ncid,"d_var",t_d_var)
NetCDF.putvar(ncid,"d_bias",t_d_bias)
NetCDF.putvar(ncid,"d_mse",t_d_mse)

NetCDF.putvar(ncid,"energy",energy)
NetCDF.putvar(ncid,"pitch",pitch)
NetCDF.putvar(ncid,"error_level",[0.05:0.05:1.0])
NetCDF.close(ncid);

In [ ]:
gaussian_full = test_dists["gaussian"]
gaussian = bilinear(gaussian_full,energy_full,pitch_full,energy,pitch)
gaussian_vec = vec(gaussian)
nr,nc = size(A)
g_x_mean = zeros(nc,20)
g_x_var = zeros(nc,20)
g_x_bias = zeros(nc,20)
g_x_mse = zeros(nc,20)

g_d_mean = zeros(20)
g_d_var = zeros(20)
g_d_bias = zeros(20)
g_d_mse = zeros(20);

In [ ]:
@showprogress for (i,n) in enumerate(0.05:0.05:1.0)
    x_samples = mf_analysis(A_full,gaussian_full,dE_full,dP_full,A,dE,dP,L1E,L1p,n)

    g_x_mean[:,i] = squeeze(mean(x_samples,2),2)
    g_x_var[:,i] = squeeze(var(x_samples,2),2)
    g_x_bias[:,i] = g_x_mean[:,i] - gaussian_vec
    g_x_mse[:,i] = g_x_bias[:,i].^2 + g_x_var[:,i]
    
    g_d_mean[i] = mean(sum(x_samples,1)*dE*dP)
    g_d_var[i] = var(sum(x_samples,1)*dE*dP)
    g_d_bias[i] = g_d_mean[i] - 1e12
    g_d_mse[i] = g_d_bias[i]^2 + g_d_var[i]
end

In [ ]:
using NetCDF
# Write synthetic data to file
fname = "data/mf_sampled_gaussian.cdf"

isfile(fname) && rm(fname)

nrows_id = NcDim("nrows",20*20)
ncols_id = NcDim("ncols",20)
nen_id = NcDim("nenergy",20)
np_id = NcDim("npitch",20)

x_mean_varid = NcVar("x_mean",[nrows_id,ncols_id])
x_var_varid = NcVar("x_var",[nrows_id,ncols_id])
x_bias_varid = NcVar("x_bias",[nrows_id,ncols_id])
x_mse_varid = NcVar("x_mse",[nrows_id,ncols_id])

d_mean_varid = NcVar("d_mean",ncols_id)
d_var_varid = NcVar("d_var",ncols_id)
d_bias_varid = NcVar("d_bias",ncols_id)
d_mse_varid = NcVar("d_mse",ncols_id)

energy_varid = NcVar("energy",nen_id)
pitch_varid = NcVar("pitch",nen_id)
elevel_varid = NcVar("error_level",ncols_id)

ncid = NetCDF.create(fname,[x_mean_varid,x_var_varid,x_bias_varid,x_mse_varid,
    d_mean_varid,d_var_varid,d_bias_varid,d_mse_varid,energy_varid,pitch_varid,elevel_varid],mode=NC_CLASSIC_MODEL)

NetCDF.putvar(ncid,"x_mean",g_x_mean)
NetCDF.putvar(ncid,"x_var",g_x_var)
NetCDF.putvar(ncid,"x_bias",g_x_bias)
NetCDF.putvar(ncid,"x_mse",g_x_mse)

NetCDF.putvar(ncid,"d_mean",g_d_mean)
NetCDF.putvar(ncid,"d_var",g_d_var)
NetCDF.putvar(ncid,"d_bias",g_d_bias)
NetCDF.putvar(ncid,"d_mse",g_d_mse)

NetCDF.putvar(ncid,"energy",energy)
NetCDF.putvar(ncid,"pitch",pitch)
NetCDF.putvar(ncid,"error_level",[0.05:0.05:1.0])
NetCDF.close(ncid);

In [ ]:
bimax_full = test_dists["bimax"]
bimax = bilinear(bimax_full,energy_full,pitch_full,energy,pitch)
bimax_vec = vec(bimax)
nr,nc = size(A)
b_x_mean = zeros(nc,20)
b_x_var = zeros(nc,20)
b_x_bias = zeros(nc,20)
b_x_mse = zeros(nc,20)

b_d_mean = zeros(20)
b_d_var = zeros(20)
b_d_bias = zeros(20)
b_d_mse = zeros(20);

In [ ]:
@showprogress for (i,n) in enumerate(0.05:0.05:1.0)
    x_samples = mf_analysis(A_full,bimax_full,dE_full,dP_full,A,dE,dP,L1E,L1p,n)

    b_x_mean[:,i] = squeeze(mean(x_samples,2),2)
    b_x_var[:,i] = squeeze(var(x_samples,2),2)
    b_x_bias[:,i] = b_x_mean[:,i] - bimax_vec
    b_x_mse[:,i] = b_x_bias[:,i].^2 + b_x_var[:,i]
    
    b_d_mean[i] = mean(sum(x_samples,1)*dE*dP)
    b_d_var[i] = var(sum(x_samples,1)*dE*dP)
    b_d_bias[i] = b_d_mean[i] - 1e12
    b_d_mse[i] = b_d_bias[i]^2 + b_d_var[i]
end

In [ ]:
using NetCDF
# Write synthetic data to file
fname = "data/mf_sampled_bimax.cdf"

isfile(fname) && rm(fname)

nrows_id = NcDim("nrows",20*20)
ncols_id = NcDim("ncols",20)
nen_id = NcDim("nenergy",20)
np_id = NcDim("npitch",20)

x_mean_varid = NcVar("x_mean",[nrows_id,ncols_id])
x_var_varid = NcVar("x_var",[nrows_id,ncols_id])
x_bias_varid = NcVar("x_bias",[nrows_id,ncols_id])
x_mse_varid = NcVar("x_mse",[nrows_id,ncols_id])

d_mean_varid = NcVar("d_mean",ncols_id)
d_var_varid = NcVar("d_var",ncols_id)
d_bias_varid = NcVar("d_bias",ncols_id)
d_mse_varid = NcVar("d_mse",ncols_id)

energy_varid = NcVar("energy",nen_id)
pitch_varid = NcVar("pitch",nen_id)
elevel_varid = NcVar("error_level",ncols_id)

ncid = NetCDF.create(fname,[x_mean_varid,x_var_varid,x_bias_varid,x_mse_varid,
    d_mean_varid,d_var_varid,d_bias_varid,d_mse_varid,energy_varid,pitch_varid,elevel_varid],mode=NC_CLASSIC_MODEL)

NetCDF.putvar(ncid,"x_mean",b_x_mean)
NetCDF.putvar(ncid,"x_var",b_x_var)
NetCDF.putvar(ncid,"x_bias",b_x_bias)
NetCDF.putvar(ncid,"x_mse",b_x_mse)

NetCDF.putvar(ncid,"d_mean",b_d_mean)
NetCDF.putvar(ncid,"d_var",b_d_var)
NetCDF.putvar(ncid,"d_bias",b_d_bias)
NetCDF.putvar(ncid,"d_mse",b_d_mse)

NetCDF.putvar(ncid,"energy",energy)
NetCDF.putvar(ncid,"pitch",pitch)
NetCDF.putvar(ncid,"error_level",[0.05:0.05:1.0])
NetCDF.close(ncid);